### Preparation of one document

In [1]:
from lxml import etree
import zipfile
from collections import OrderedDict
import os

SEED=42
MODEL_PATH='model/'
MODEL_FILE_NAME='ner_model.ckpt'

def find_between(s,first,last):
    try:
        start=s.index(first)+len(first)
        end=s.index(last,start)
        if len(s[start:end]) >= 1:
            return s[start:end]
        else:
            return '!'
    except ValueError:
        return '!'

### Comments

In [2]:
import re
docxZip=zipfile.ZipFile('ner_test.docx')
textsXML=docxZip.read('word/document.xml')
xml_str=textsXML.decode('utf-8')

In [3]:
s=xml_str
proccess=1
needed_text_for_docx=[]
while proccess:
    sub_str=find_between(s, 'commentRangeStart', 'commentRangeEnd')
    sub_str_final = find_between(sub_str, '<w:t>', '</w:t>')
    if sub_str_final=='!':
        sub_str_final=find_between(sub_str, '<w:t xml:space="preserve">', '</w:t>')
    if sub_str == '!':
        break
    s=s.replace(sub_str,'',1)
    s=s.replace('commentRangeStartcommentRangeEnd', '', 1)
    sub_str_final=sub_str_final.lstrip().rstrip()
    needed_text_for_docx.append(sub_str_final)

### Text of comments in Word

In [4]:
import re
docxZip=zipfile.ZipFile('ner_test.docx')
textsXML=docxZip.read('word/comments.xml')
xml_str=textsXML.decode('utf-8')

In [5]:
classes_for_docx=[]
s=xml_str
sub_str_all=''
proccess=1
while proccess:
    sub_str_all=''
    sub_str=find_between(s, '<w:comment w:', '/w:comment>')
    if sub_str=='!':
        break
    s=s.replace(sub_str,'',1)
    s=s.replace('<w:comment w:/w:comment>', '', 1)
    while proccess:
        sub_str_one=find_between(sub_str, '<w:t>', '</w:t>')
        if sub_str_one == '!':
            break
        sub_str=sub_str.replace('<w:t>'+sub_str_one+'</w:t>', '', 1)
        sub_str=sub_str.lstrip().rstrip()
        sub_str_all+=sub_str_one
    classes_for_docx.append(sub_str_all)

In [6]:
needed_text_for_docx

['Перейдем теперь', 'Во-первых']

In [7]:
classes_for_docx

['Test1', 'Test2']

### Preprocessing for NER

In [8]:
from docx import Document

In [9]:
import docx

def para2text(p):
    rs=p._element.xpath('.//w:t')
    return u" ".join([r.text for r in rs])

def getText(filename):
    doc=docx.Document(filename)
    fullText=[]
    for para in doc.paragraphs:
        #smartTag
        #fullText.append(para2text[para])
        #without smartTag
        fullText.append(para.text)
    return '\n'.join(fullText)

In [10]:
#Document to sentences
# -*- coding: utf-8 -*-
import re
alphabets= "([А-Яа-я])"
prefixes = "(ст|Рф|Гз)[.]"
suffixes = "(ст|Рф|Гз)"
starters = "(ст|Рф|Гз)"
acronyms = "([А-Я][.][А-Я][.](?:[А-Я][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences_rus(text):
    digits = "([0-9])"
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text) 
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "'" in text: text = text.replace("."","".")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    
    return sentences

In [11]:
text=getText('ner_test.docx')

In [12]:
split_text=split_into_sentences_rus(text)

In [13]:
sentences=[] #all sentences
counter_of_comments=0
for text in split_text:
    checker='-' #is it in sentence or not
    sentence=[] #for every sentence
    tokens=[] #for every sentence
    tags=[] #for every sentence
    for index in range(counter_of_comments, len(needed_text_for_docx)):
        checker='-'
        if needed_text_for_docx[index] in text: #write comments
            checker='+'
            text_before=text[:text.find(needed_text_for_docx[index])].split()
            if len(text_before) > 1:
                for i in range(len(text_before)):
                    tokens.append(text_before[i])
                    tags.append('O')
            else:
                try:
                    tokens.append(text_before[0])
                    tags.append('O')
                except IndexError:
                    pass
            text = text[text.find(needed_text_for_docx[index]):]
            #append comment and text
            comment = needed_text_for_docx[index].split()
            if len(comment)==1:
                tokens.append(comment[i])
                tags.append('B-'+classes_for_docx[index])
            else:
                for i in range(len(comment)):
                    tokens.append(comment[i])
                    if i == 0:
                        tags.append('B-'+classes_for_docx[index])
                    else:
                        tags.append('I-'+classes_for_docx[index])
            text = text.replace(needed_text_for_docx[index], '',1)
        else:
            if index==counter_of_comments:
                checker='-'
                break
            else: #add comment to the end
                checker='+'
                text=text.split()
                if len(text) > 1:
                    for i in range(len(text)):
                        tokens.append(text[i])
                        tags.append('O')
                else:
                    try:
                        tokens.append(text[0])
                        tags.append('O')
                    except IndexError:
                        pass
                break
    counter_of_comments = index
    if checker == '-':
        text=text.split()
        if len(text) > 1:
            for i in range(len(text)):
                tokens.append(text[i])
                tags.append('O')
        else:
            tokens.append(text[0])
            tags.append('O')
    sentence.append((tokens,tags))
    sentences.append((tokens,tags))

In [14]:
#Add in the end
for i in range(len(sentences)):
    sentences[i][0].append('.')
    sentences[i][1].append('O')

In [15]:
sentences

[(['Рассмотренные',
   'ранее',
   'когерентные',
   'состояния',
   'являются',
   'очень',
   'важным,',
   'но,',
   'тем',
   'не',
   'менее,',
   'частным',
   'случаем',
   'из',
   'класса',
   'состояний,',
   'которые',
   'минимизируют',
   'соотношение',
   'неопределенностей.',
   '.'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['Действительно,',
   'в',
   'когерентном',
   'состоянии',
   'флуктуации',
   'координаты',
   'и',
   'импульса',
   'равны:',
   'Перейдем',
   'теперь',
   'к',
   'общему',
   'случаю',
   'и,',
   'прежде',
   'всего,',
   'уточним',
   'постановку',
   'вопроса.',
   '.'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-Test1',
   'I-Test1',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['А',
   'именно,',
   'нас',
   'будут',
   'интересовать',